# Creating interactive maps with Leaflet and Folium 

## Setup

#### Dependencies

Install NodeJS
```
conda install -c conda-forge nodejs 
```

Install Folium
```
conda install -c conda-forge folium
jupyter nbextension enable vega --py --sys-prefix
```

Install Google Earth API
```
pip install google-api-python-client
pip install pyCrypto
pip install earthengine-api
```

Install GeoJSON extension
```
jupyter labextension install @jupyterlab/geojson-extension
```

#### Authenticate to Google Earth Engine

Verify your Google Earth Engine (GEE) Account - this should be a one time step.
```
earthengine authenticate
```

Convert GEE kml files to geojson
```
pip install kml2geojson
```

# Data Download

Data should have been downloaded using the previous notebook `NEON_api_get_python_lidar.ipynb`

## File Types

|types|Source|Description|
|-----|------|-----------|
|\*.laz|NEON|Discrete lidar|
|\*.geojson|any|point, line, shapes|
|\*.kml,\*.kmz|any|point, line, shapes|
|\*.tif|any|orthophotography|

## Begin Scripting with Python3

In [ ]:
# Initialize GEE
import ee 
ee.Initialize()

# Import Data Science Packages
import pandas as pd
import json
import sys

# Import Folium Leaflet maps
import folium
print(folium.__version__)
sys.path.append('..')
print (folium.__file__)
print (folium.__version__)

### Create definition files for working with Google Earth Engine

In [ ]:
def folium_gee_map(image,vis_params=None,folium_kwargs={}):
    """
    Function to view Google Earth Engine tile layer as a Folium map.
    
    Parameters
    ----------
    image : Google Earth Engine Image.
    vis_params : Dict with visualization parameters.
    folium_kwargs : Keyword args for Folium Map.
    """
    
    # Get the MapID and Token after applying parameters
    image_info = image.getMapId(vis_params)
    mapid = image_info['mapid']
    token = image_info['token']
    folium_kwargs['attr'] = ('Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a> ')
    folium_kwargs['tiles'] = "https://earthengine.googleapis.com/map/%s/{z}/{x}/{y}?token=%s"%(mapid,token)
    
    return folium.Map(**folium_kwargs)

def folium_gee_layer(folium_map,image,vis_params=None,folium_kwargs={}):
    """
    Function to add Google Earch Engine tile layer as a Folium layer.
    
    Parameters
    ----------
    folium_map : Folium map to add tile to.
    image : Google Earth Engine Image.
    vis_params : Dict with visualization parameters.
    folium_kwargs : Keyword args for Folium Map.
    """
    
    # Get the MapID and Token after applying parameters
    image_info = image.getMapId(vis_params)
    mapid = image_info['mapid']
    token = image_info['token']
    folium_kwargs['attr'] = ('Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a> ')
    folium_kwargs['tiles'] = "https://earthengine.googleapis.com/map/%s/{z}/{x}/{y}?token=%s"%(mapid,token)
    
    layer = folium.TileLayer(**folium_kwargs)
    layer.add_to(folium_map)


Get the Boundary files from the SRER flight (`../L1/LasBoundary`) 

GeoJSON creation with `k2g`

In [11]:
!cd
!mkdir srer_boundary
!iget -KPQbrvf /iplant/home/shared/srer-wgew/data/SRER/L1/LasBoundary /home/tyson_swetnam/srer_boundary

D- /home/tyson_swetnam/srer_boundary :
0/128 -  0.00% of files done   0.000/20.066 MB -  0.00% of file sizes done
Processing NEON_D14_SRER_DP1_L001-1_2017082413_unclassified_point_cloud_boundary.kml - 0.212 MB   2018-04-23.09:58:38
   NEON_D14_SRER_DP1_L001-1_       0.212 MB | 1.265 sec | 0 thr |  0.168 MB/s
1/128 -  0.78% of files done   0.212/20.066 MB -  1.06% of file sizes done
Processing NEON_D14_SRER_DP1_L001-2_2017082419_unclassified_point_cloud_boundary.kml - 0.249 MB   2018-04-23.09:58:40
   NEON_D14_SRER_DP1_L001-2_       0.249 MB | 0.366 sec | 0 thr |  0.681 MB/s
2/128 -  1.56% of files done   0.461/20.066 MB -  2.30% of file sizes done
Processing NEON_D14_SRER_DP1_L001-3_2017082514_unclassified_point_cloud_boundary.kml - 0.063 MB   2018-04-23.09:58:40
   NEON_D14_SRER_DP1_L001-3_       0.063 MB | 1.214 sec | 0 thr |  0.052 MB/s
3/128 -  2.34% of files done   0.524/20.066 MB -  2.61% of file sizes done
Processing NEON_D14_SRER_DP1_L002-1_2017082413_unclassified_point_cloud_b

In [ ]:
!k2g /home/tyson_swetnam/srer_boundary/full_boundary_all_files.kml /home/tyson_swetnam/srer_boundary

In [ ]:
!k2g /home/tyson_swetnam/srer_boundary/full_boundary.kml /home/tyson_swetnam/srer_boundary

In [ ]:
# import geo json data
full_boundary_all = json.load(open('/home/tyson_swetnam/srer_boundary/full_boundary_all_files.geojson'))
full_boundary = json.load(open('/home/tyson_swetnam/srer_boundary/full_boundary.geojson'))

In [ ]:
# Get an area to look at
lat = 31.8260
lon = -110.865
zoom_start=12

# Open Street Map Base
m = folium.Map(location=[lat, lon], tiles="OpenStreetMap", zoom_start=zoom_start)

# Add polygon outline of SRER
full_boundary = json.load(open('/home/tyson_swetnam/srer_boundary/full_boundary.geojson'))
folium.GeoJson(full_boundary).add_to(m)

# Add GEE Terrain Layer
image = ee.Image('srtm90_v4')
vis_params = {'min':0.0, 'max':3000, 'palette':'00FFFF,0000FF'}
folium_gee_layer(m,image,vis_params=vis_params,folium_kwargs={'overlay':True,'name':'SRTM'})

# Create a reference to the LANDSAT 8 image collection
l8 = ee.ImageCollection('LANDSAT/LC8_L1T_TOA')
# Filter the LANDSAT collection down to a eight month period
filtered = l8.filterDate('2017-01-01', '2017-08-31');
# Use the mosaic reducer, to select the most recent pixel in areas of overlap
l8_image = filtered.median()
l8_vis_params = {'min': 0, 'max':0.3, 'bands':'B4,B3,B2'}
folium_gee_layer(m,l8_image,l8_vis_params,folium_kwargs={'overlay':True,'name':'LANDSAT'})

# Create a reference to the 2017 NEON orthophotography image collection
#neon = ee.ImageCollection('users/gponce/usda_ars/image_collections/neon_srer_2017_rgb')
#neon_image = neon.median()
#n eon_vis_params = {'min':0, 'max':255, 'bands':'b1,b2,b3'}
#folium_gee_layer(m,neon_image,neon_vis_params,folium_kwargs={'overlay':True,'name':'NEON'})

# Create a reference to the 2016 sUAS orthophotography imager collection

m.add_child(folium.LayerControl())
m

In [ ]:
import gdal as gdal
import osr as osr

gdal.UseExceptions()

fname = '/home/tswetnam/wgew/LH_20agl_9Oct2017_georef.tif'

ds = gdal.Open(fname)
data = ds.ReadAsArray()
gt = ds.GetGeoTransform()
proj = ds.GetProjection()

inproj = osr.SpatialReference()
inproj.ImportFromWkt(proj)

print(inproj)

In [6]:
!iget -KPQvrf /iplant/home/shared/srer-wgew/data/wgew/sfm_2017/LH_20agl_9Oct2017_georef.tif

0/1 -  0.00% of files done   0.000/6512.374 MB -  0.00% of file sizes done
Processing LH_20agl_9Oct2017_georef.tif - 6512.374 MB   2018-04-19.13:22:11
LH_20agl_9Oct2017_georef.tif - 840.000/6512.374 MB - 12.90% done   2018-04-19.13:22:23
LH_20agl_9Oct2017_georef.tif - 1880.000/6512.374 MB - 28.87% done   2018-04-19.13:22:32
LH_20agl_9Oct2017_georef.tif - 2680.000/6512.374 MB - 41.15% done   2018-04-19.13:22:39
LH_20agl_9Oct2017_georef.tif - 3520.000/6512.374 MB - 54.05% done   2018-04-19.13:22:46
LH_20agl_9Oct2017_georef.tif - 4080.000/6512.374 MB - 62.65% done   2018-04-19.13:22:53
LH_20agl_9Oct2017_georef.tif - 4927.023/6512.374 MB - 75.66% done   2018-04-19.13:22:59
LH_20agl_9Oct2017_georef.tif - 5595.117/6512.374 MB - 85.92% done   2018-04-19.13:23:05
LH_20agl_9Oct2017_georef.tif - 6230.234/6512.374 MB - 95.67% done   2018-04-19.13:23:10
LH_20agl_9Oct2017_georef.tif - 6465.351/6512.374 MB - 99.28% done   2018-04-19.13:23:11
LH_20agl_9Oct2017_georef.tif - 6505.351/6512.374 MB - 99.8

In [43]:
!gdalwarp -t_srs EPSG:4326 -r near /home/tswetnam/wgew/LH_20agl_9Oct2017_georef.tif /home/tswetnam/wgew/LH_20agl_9Oct2017_georef2.tif

Processing input file /home/tswetnam/wgew/LH_20agl_9Oct2017_georef.tif.
Using internal nodata values (e.g. 256) for image /home/tswetnam/wgew/LH_20agl_9Oct2017_georef.tif.
ERROR 1: Too many points (441 out of 441) failed to transform, unable to compute output bounds.
Warning 1: Unable to compute source region for output window 0,0,36805,31362, skipping.
0...10...20...30...40...50...60...70...80...90...100 - done.


In [44]:
!gdal_translate -of mbtiles  /home/tswetnam/wgew/LH_20agl_9Oct2017_georef2.tif  /home/tswetnam/wgew/LH_20agl_9Oct2017_georef.mbtiles
!gdaladdo -r nearest /home/tswetnam/wgew/LH_20agl_9Oct2017_georef.mbtiles 2 4 8 16

Input file size is 36805, 31362
ERROR 6: zoom_level > 22 not supported
0...10...20...30...40...50...60...70...80...90...100 - done.
ERROR 6: zoom_level > 22 not supported
